In [1]:
import mujoco
import mujoco.viewer
from pathlib import Path
import numpy as np
import time

WIN_MODEL_PATH = r"C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\models\scene_right_piano_hand.xml"
UNIVERSAL_MODEL_PATH = Path(WIN_MODEL_PATH)

print(str(UNIVERSAL_MODEL_PATH))
model = mujoco.MjModel.from_xml_path(str(UNIVERSAL_MODEL_PATH))
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)


C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\models\scene_right_piano_hand.xml


ValueError: mjParseXML: resource not found via provider or OS filesystem: 'C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\models\scene_right_piano_hand.xml'

In [ ]:
end_effector_site_names = ["fingertip_index", "fingertip_middle"]


end_effector_site_ids = [mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, name) for name in end_effector_site_names]
print(end_effector_site_ids)


jacp = np.zeros((3, model.nv), dtype=float) # translational Jacobian
jacr = np.zeros((3, model.nv), dtype=float) # translational Jacobian

jacp_combined = np.zeros((3*len(end_effector_site_ids), model.nv), dtype=float)   

for id in end_effector_site_ids:
    print("Site:", mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_SITE, id))
    
    print("Translational Jacobian: ")
    mujoco.mj_jacSite(model, data, jacp, jacr, id)
    print(jacp)
    print(jacp.shape)
    print()





[1, 2]
Site: fingertip_index
Translational Jacobian: 
[[-2.22044605e-16  0.00000000e+00  1.00000000e+00 -3.30000000e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.99000000e-01
   0.00000000e+00 -7.00000000e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00 -2.22044605e-16  0

In [ ]:
def get_multifinger_jacobian(model, data, site_names):
    jacp = np.zeros((3, model.nv))
    jacr = np.zeros((3, model.nv))
    
    jac_combined = np.zeros((6*len(site_names), model.nv))
    
    for i, site_name in enumerate(site_names):
        mujoco.mj_jacSite(model, data, jacp, jacr, mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name))
        # jac_combined[6*i:6*(i+1), :] = np.vstack((jacp, jacr))
        
        jac_combined[6*i:6*(i+1), :] = np.vstack((jacp, jacr))
    
    return jac_combined

def get_error(model, data, site_names, target_names):
    
    error = np.zeros(6*len(site_names)) # [site_1_pos_error, site_1_rot_error, ..., site_N_pos_error, site_N_rot_error]
    
    for i, site_name in enumerate(site_names):
        site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
        target_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, target_names[i])
        
        # preallocate arrays for a single site computation
        error_pos = error[6*i:6*i+3]
        error_rot = error[6*i+3:6*(i+1)]

        site_quat = np.zeros(4)
        site_quat_conj = np.zeros(4)
        
        target_quat = np.zeros(4)
        
        error_quat = np.zeros(4)
        
        # positional error
        error_pos[:] = data.geom(target_id).xpos - data.site(site_id).xpos

        
        # orientation error
        mujoco.mju_mat2Quat(site_quat, data.site(site_id).xmat)
        mujoco.mju_mat2Quat(target_quat, data.geom(target_id).xmat)
        
        mujoco.mju_negQuat(site_quat_conj, site_quat)
        mujoco.mju_mulQuat(error_quat, target_quat, site_quat_conj)
        mujoco.mju_quat2Vel(error_rot, error_quat, 1.0)
        # print(np.hstack((error_pos, error_rot)))
        # error[6*i:6*(i+1)] = np.hstack((error_pos, error_rot))
        
    return error

def apply_joint_limits(model, q):
    for i in range(len(q)):
        q[i] = max(model.jnt_range[i][0], 
               min(q[i], model.jnt_range[i][1]))


def solve_multifinger_ik(model, data, site_names, target_names, step_size=0.5, tol=1e-6, max_iter=1000):
    """Returns the joint angles that place the fingertips (named by site_names) at the target poses.
    Args:
        model (_type_): _description_
        data (_type_): _description_
        site_names (_type_): _description_
        target_poses (_type_): _description_

    Returns:
        _type_: _description_
    """
    assert len(site_names) == len(target_names)
    
    jac = np.zeros((6, model.nv)) 
    
    # compute error
    error = get_error(model, data, site_names, target_names)
    
    # TODO: write function for error
    # solution loop
    iter = 0
    while (np.linalg.norm(error) > tol and iter<=max_iter):

        jac = get_multifinger_jacobian(model, data, site_names)
        dq = np.linalg.pinv(jac) @ error
        data.qpos += step_size*dq
        iter+=1
        
        apply_joint_limits(model, data.qpos)
        mujoco.mj_forward(model, data)
        
       # compute error
        error = get_error(model, data, site_names, target_names)
        print(np.linalg.norm(error))
        

    
    return data.qpos, np.linalg.norm(error), iter

In [ ]:
# end_effector_site_names = ["fingertip_index", "fingertip_middle"]
# target_names = ["keymarker_1", "keymarker_2"]
end_effector_site_names = ["fingertip_index"]
target_names = ["keymarker_1"]

# target_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, target_names[0])
# print(target_id)

qpos, error, iter = solve_multifinger_ik(model, data, end_effector_site_names, target_names)
data.qpos = qpos
mujoco.mj_forward(model, data)
mujoco.viewer.launch(model, data)
print(iter)

2.079021486168618
2.066311778769821
2.062175356138841
2.0606716682605266
2.0600613877493985
2.0597909939739094
2.059663962242081
2.059602176107769
2.059571545848687
2.0595562075491056
2.0595484868607192
2.0595445902852365
2.05954262107502
2.0595416252224985
2.0595411214364048
2.0595408665351886
2.0595407375514134
2.059540672280908
2.0595406392509403
2.0595406225360384
2.059540614077377
2.0595406097968194
2.059540607630615
2.0595406065343935
2.0595406059796435
2.059540605698909
2.0595406055568413
2.059540605484947
2.059540605448565
2.0595406054301537
2.0595406054208367
2.0595406054161214
2.0595406054137353
2.0595406054125283
2.059540605411917
2.0595406054116077
2.0595406054114513
2.0595406054113723
2.059540605411332
2.059540605411312
2.0595406054113012
2.0595406054112964
2.0595406054112937
2.0595406054112924
2.059540605411292
2.0595406054112915
2.059540605411291
2.059540605411291
2.059540605411291
2.059540605411291
2.059540605411291
2.059540605411291
2.059540605411291
2.059540605411291


In [ ]:
model.geom("keymarker_1").pos
target_poses = np.array([model.geom(name).pos for name in target_names])
print(target_poses)
model.geom("keymarker_2")
# data.geom("keymarker_1")


[[ 0.4601 -0.43    0.12  ]]


<_MjModelGeomViews
  bodyid: array([0])
  conaffinity: array([0])
  condim: array([3])
  contype: array([0])
  dataid: array([-1])
  friction: array([1.e+00, 5.e-03, 1.e-04])
  gap: array([0.])
  group: array([2])
  id: 2
  margin: array([0.])
  matid: array([4])
  name: 'keymarker_2'
  pos: array([0.3701, 0.3   , 0.14  ])
  priority: array([0])
  quat: array([1., 0., 0., 0.])
  rbound: array([0.02])
  rgba: array([0.5, 0.5, 0.5, 1. ], dtype=float32)
  sameframe: array([0], dtype=uint8)
  size: array([0.02, 0.  , 0.  ])
  solimp: array([9.0e-01, 9.5e-01, 1.0e-03, 5.0e-01, 2.0e+00])
  solmix: array([1.])
  solref: array([0.02, 1.  ])
  type: array([2])
  user: array([], dtype=float64)
>